In [ ]:
import cv2
import glob
import numpy as np
import pickle
import os

# 棋盘规格
BOARD_RAW = 9
BOARD_COL = 6

# 图片保存路径
IMG_SAVE_PATH = "img/"
IMG_RESULT_PATH = "result/"

if __name__ == '__main__':
    # 创建result文件夹（如果不存在）
    if not os.path.exists(IMG_RESULT_PATH):
        os.makedirs(IMG_RESULT_PATH)
        
    obj_p = np.zeros((BOARD_RAW*BOARD_COL, 3), np.float32)

    # np.mgrid[0:raw, 0:col]的shape为(2, 9, 6)转置后为(6, 9, 2)，reshape后为(6*9, 2)
    # obj_p[:, :2]===>obj_p[:, 0] and obj_p[:, 1]
    obj_p[:, :2] = np.mgrid[0:BOARD_RAW, 0:BOARD_COL].T.reshape(-1, 2)

    obj_points = []
    img_points = []

    images = glob.glob(IMG_SAVE_PATH + '*.jpg')
    images = sorted(set(images), key=lambda x: int(os.path.splitext(os.path.basename(x))[0]))
    
    print(f"找到 {len(images)} 张图片")
    successful_detections = 0
    
    for name in images:
        print(f'正在处理: {os.path.basename(name)}')
        img = cv2.imread(name)
        
        if img is None:
            print(f"  ✗ 无法读取图片: {name}")
            continue
            
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # 寻找角点
        ret, corners = cv2.findChessboardCorners(gray, (BOARD_RAW, BOARD_COL), None)
        
        # 重要：只有在成功找到角点时才进行后续处理
        if ret and corners is not None:
            criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
            sub_corners = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)

            obj_points.append(obj_p)
            img_points.append(sub_corners)

            # 在彩色图像上绘制角点
            img_with_corners = cv2.drawChessboardCorners(img.copy(), (BOARD_RAW, BOARD_COL), sub_corners, ret)
            
            # 保存带角点的图像
            result_filename = 'detected_' + os.path.basename(name)
            cv2.imwrite(IMG_RESULT_PATH + result_filename, img_with_corners)
            cv2.waitKey(500)
            
            successful_detections += 1
        else:
            print("未检测到角点")

    cv2.destroyAllWindows()
    
    print(f"\n处理完成: {successful_detections}/{len(images)} 张图片成功检测到角点")

    if successful_detections == 0:
        print("错误：没有成功检测到任何角点！")
        exit()

    print("\n开始相机标定...")

    # 标定结果：相机的内参数矩阵，畸变系数，旋转矩阵和平移向量
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, gray.shape[::-1], None, None)
    
    print('标定成功!')
    print('重投影误差 (越小越好):', ret)
    print('\n相机内参矩阵:')
    print(mtx)
    print('\n畸变系数:')
    print(dist)

    # 保存参数
    cal_parameter = {'ret': ret, 'mtx': mtx, 'dist': dist, 'rvecs': rvecs, 'tvecs': tvecs}
    
    # 文件保存
    with open("camera_parameters.pkl", "wb") as f:
        pickle.dump(cal_parameter, f)
    print("\n参数保存成功: camera_parameters.pkl")

    print("\n标定完成！")

In [ ]:
import cv2
import pickle
import glob
import os

# 创建输出文件夹
if not os.path.exists('processed'):
    os.makedirs('processed')

# 读取相机参数
f = pickle.load(open('camera_parameters.pkl', 'rb'))
mtx, dist = f['mtx'], f['dist']

# 处理所有图片
images = glob.glob('img/*.jpg') + glob.glob('img/*.png')
print(f"找到 {len(images)} 张图片")

for img_path in images:
    # 读取图片
    img = cv2.imread(img_path)
    
    # 畸变校正
    corrected = cv2.undistort(img, mtx, dist, None, mtx)
    
    # 保存到processed文件夹
    filename = os.path.basename(img_path)
    output_path = f'processed/{filename}'
    cv2.imwrite(output_path, corrected)
    
    print(f"✓ {filename}")

In [ ]:
import pickle
import numpy as np

# 读取相机标定参数
f = pickle.load(open('camera_parameters.pkl', 'rb'))
mtx, dist = f['mtx'], f['dist']

# 提取相机内参
fx = mtx[0, 0]  # 焦距x
fy = mtx[1, 1]  # 焦距y
cx = mtx[0, 2]  # 主点x
cy = mtx[1, 2]  # 主点y

# 提取畸变系数 (dist是2D数组)
k1 = dist[0][0]    # 径向畸变系数1
k2 = dist[0][1]    # 径向畸变系数2
p1 = dist[0][2]    # 切向畸变系数1
p2 = dist[0][3]    # 切向畸变系数2
k3 = dist[0][4] if dist.shape[1] > 4 else 0  # 径向畸变系数3

print("相机内参:")
print(f"fx = {fx:.6f}")
print(f"fy = {fy:.6f}")
print(f"cx = {cx:.6f}")
print(f"cy = {cy:.6f}")
print()
print("畸变系数:")
print(f"k1 = {k1:.6f}")
print(f"k2 = {k2:.6f}")
print(f"k3 = {k3:.6f}")
print(f"p1 = {p1:.6f}")
print(f"p2 = {p2:.6f}")

# 四次多项式系数计算

将像素重映射表达式展开为标准的四次多项式形式：
- x' = a0 + a1*x + a2*y + a3*x² + a4*x*y + a5*y² + a6*x³ + a7*x²*y + a8*x*y² + a9*y³ + a10*x⁴ + a11*x³*y + a12*x²*y² + a13*x*y³ + a14*y⁴
- y' = b0 + b1*x + b2*y + b3*x² + b4*x*y + b5*y² + b6*x³ + b7*x²*y + b8*x*y² + b9*y³ + b10*x⁴ + b11*x³*y + b12*x²*y² + b13*x*y³ + b14*y⁴

In [ ]:
import sympy as sp
from sympy import symbols, expand, collect

# 定义符号变量
x, y = symbols('x y')

# 相机参数 (实际数值)
fx_val = mtx[0][0]  # 1149.45237624
fy_val = mtx[1][1]  # 1147.15555982  
cx_val = mtx[0][2]  # 925.88740734
cy_val = mtx[1][2]  # 535.47098450

k1_val = dist[0][0]  # -0.35771687
k2_val = dist[0][1]  # 0.18385165
p1_val = dist[0][2]  # -0.00058972
p2_val = dist[0][3]  # 0.00059129
k3_val = dist[0][4]  # -0.05944617

# 定义归一化坐标
u = (x - cx_val) / fx_val
v = (y - cy_val) / fy_val
r_squared = u**2 + v**2

# 径向畸变校正因子 (截断到r⁴，因为要保持最高四次)
radial_factor = 1 + k1_val * r_squared + k2_val * r_squared**2

# 切向畸变
tangential_x = 2 * p1_val * u * v + p2_val * (r_squared + 2 * u**2)
tangential_y = p1_val * (r_squared + 2 * v**2) + 2 * p2_val * u * v

# 校正后的归一化坐标
u_corrected = u * radial_factor + tangential_x
v_corrected = v * radial_factor + tangential_y

# 转换回像素坐标
x_prime = u_corrected * fx_val + cx_val
y_prime = v_corrected * fy_val + cy_val

# 展开到四次项并截断
x_prime_expanded = expand(x_prime)
y_prime_expanded = expand(y_prime)

def truncate_to_degree_4(expr, vars):
    """截断表达式到四次项"""
    result = 0
    for term in expr.as_ordered_terms():
        degree = sum(term.as_poly(*vars).degree_list()) if hasattr(term.as_poly(*vars), 'degree_list') else 0
        if degree <= 4:
            result += term
    return result

x_prime_truncated = truncate_to_degree_4(x_prime_expanded, [x, y])
y_prime_truncated = truncate_to_degree_4(y_prime_expanded, [x, y])

# 提取系数函数
def extract_coefficients_degree_4(expr, x_var, y_var):
    """提取四次多项式的15个系数"""
    coeffs = [0] * 15  # 15个系数
    
    # 定义项的顺序: 1, x, y, x², xy, y², x³, x²y, xy², y³, x⁴, x³y, x²y², xy³, y⁴
    terms = [
        1,                    # a0: 常数项
        x_var,                # a1: x
        y_var,                # a2: y  
        x_var**2,             # a3: x²
        x_var*y_var,          # a4: xy
        y_var**2,             # a5: y²
        x_var**3,             # a6: x³
        x_var**2*y_var,       # a7: x²y
        x_var*y_var**2,       # a8: xy²
        y_var**3,             # a9: y³
        x_var**4,             # a10: x⁴
        x_var**3*y_var,       # a11: x³y
        x_var**2*y_var**2,    # a12: x²y²
        x_var*y_var**3,       # a13: xy³
        y_var**4              # a14: y⁴
    ]
    
    # 将表达式转换为多项式并提取系数
    poly = sp.poly(expr, x_var, y_var)
    
    for i, term in enumerate(terms):
        term_poly = sp.poly(term, x_var, y_var)
        if term_poly.monoms():
            monom = term_poly.monoms()[0]
            if monom in poly.monoms():
                coeffs[i] = float(poly.coeff_monomial(monom))
    
    return coeffs

# 提取x'和y'的系数
x_coeffs = extract_coefficients_degree_4(x_prime_truncated, x, y)
y_coeffs = extract_coefficients_degree_4(y_prime_truncated, x, y)

# 应用放大倍数: 0次项×4, 1次项×15, 2次项×25, 3次项×36, 4次项×46
scale_factors = [4, 15, 25, 36, 46]
term_degrees = [0, 1, 1, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4]

x_coeffs_scaled = [coeff * scale_factors[degree] for coeff, degree in zip(x_coeffs, term_degrees)]
y_coeffs_scaled = [coeff * scale_factors[degree] for coeff, degree in zip(y_coeffs, term_degrees)]

print("\n// X方向映射系数 (x')")
print("// x' = a0 + a1*x + a2*y + a3*x² + a4*x*y + a5*y² + a6*x³ + a7*x²*y + a8*x*y² + a9*y³ + a10*x⁴ + a11*x³*y + a12*x²*y² + a13*x*y³ + a14*y⁴")

term_names = ['a0', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8', 'a9', 'a10', 'a11', 'a12', 'a13', 'a14']
term_descriptions = ['常数', 'x', 'y', 'x²', 'xy', 'y²', 'x³', 'x²y', 'xy²', 'y³', 'x⁴', 'x³y', 'x²y²', 'xy³', 'y⁴']

for i, (name, desc, coeff, degree) in enumerate(zip(term_names, term_descriptions, x_coeffs_scaled, term_degrees)):
    fixed_point = int(coeff * (2**16)) if abs(coeff) > 1e-15 else 0
    scale = scale_factors[degree]
    # 处理负数的补码表示
    if fixed_point < 0:
        fixed_point = fixed_point & 0xFFFFFFFF
    print(f"{name}({desc:4s}) = 0x{fixed_point:08X}  // {coeff:15.8e} [×{scale}]")

print("\n// Y方向映射系数 (y')")
print("// y' = b0 + b1*x + b2*y + b3*x² + b4*x*y + b5*y² + b6*x³ + b7*x²*y + b8*x*y² + b9*y³ + b10*x⁴ + b11*x³*y + b12*x²*y² + b13*x*y³ + b14*y⁴")

term_names_y = ['b0', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9', 'b10', 'b11', 'b12', 'b13', 'b14']

for i, (name, desc, coeff, degree) in enumerate(zip(term_names_y, term_descriptions, y_coeffs_scaled, term_degrees)):
    fixed_point = int(coeff * (2**16)) if abs(coeff) > 1e-15 else 0
    scale = scale_factors[degree]
    # 处理负数的补码表示
    if fixed_point < 0:
        fixed_point = fixed_point & 0xFFFFFFFF
    print(f"{name}({desc:4s}) = 0x{fixed_point:08X}  // {coeff:15.8e} [×{scale}]")

print("\n" + "="*70)
print("纯16进制系数值 (便于Verilog代码使用):")
print("="*70)

print("\n// X方向系数")
for i, (name, coeff) in enumerate(zip(term_names, x_coeffs_scaled)):
    fixed_point = int(coeff * (2**16)) if abs(coeff) > 1e-15 else 0
    if fixed_point < 0:
        fixed_point = fixed_point & 0xFFFFFFFF
    print(f"localparam [31:0] {name.upper()}_X = 32'h{fixed_point:08X};")

print("\n// Y方向系数")
for i, (name, coeff) in enumerate(zip(term_names_y, y_coeffs_scaled)):
    fixed_point = int(coeff * (2**16)) if abs(coeff) > 1e-15 else 0
    if fixed_point < 0:
        fixed_point = fixed_point & 0xFFFFFFFF
    print(f"localparam [31:0] {name.upper()}_Y = 32'h{fixed_point:08X};")

print("\n计算完成！")

开始计算四次多项式系数...

FPGA实现用系数 - 16进制定点数 (16位小数位)

// X方向映射系数 (x')
// x' = a0 + a1*x + a2*y + a3*x² + a4*x*y + a5*y² + a6*x³ + a7*x²*y + a8*x*y² + a9*y³ + a10*x⁴ + a11*x³*y + a12*x²*y² + a13*x*y³ + a14*y⁴
a0(常数  ) = 0x0280990A  //  6.40597819e+02 [×4]
a1(x   ) = 0x000BA07B  //  1.16268911e+01 [×15]
a2(y   ) = 0xFFFF90A1  // -4.35048524e-01 [×15]
a3(x²  ) = 0xFFFFFE66  // -6.27001794e-03 [×25]
a4(xy  ) = 0xFFFFFDB7  // -8.93966876e-03 [×25]
a5(y²  ) = 0xFFFFFF76  // -2.11916517e-03 [×25]
a6(x³  ) = 0x00000001  //  2.49391313e-05 [×36]
a7(x²y ) = 0x00000001  //  2.26474242e-05 [×36]
a8(xy² ) = 0x00000001  //  1.63692282e-05 [×36]
a9(y³  ) = 0x00000000  //  7.57940120e-06 [×36]
a10(x⁴  ) = 0x00000000  // -2.24279524e-08 [×46]
a11(x³y ) = 0x00000000  // -1.04182485e-08 [×46]
a12(x²y²) = 0x00000000  // -2.70214225e-08 [×46]
a13(xy³ ) = 0x00000000  // -1.04600088e-08 [×46]
a14(y⁴  ) = 0x00000000  // -4.52162241e-09 [×46]

// Y方向映射系数 (y')
// y' = b0 + b1*x + b2*y + b3*x² + b4*x*y + b5*y² + b6*x³ +

In [9]:
import csv

# 准备CSV数据：15行2列
csv_data = []

for i in range(15):
    # X方向系数（左列）
    x_coeff = x_coeffs_scaled[i]
    x_fixed_point = int(x_coeff * (2**16)) if abs(x_coeff) > 1e-15 else 0
    if x_fixed_point < 0:
        x_fixed_point = x_fixed_point & 0xFFFFFFFF
    x_hex_str = f"{x_fixed_point:x}" if x_fixed_point != 0 else "0"
    
    # Y方向系数（右列）
    y_coeff = y_coeffs_scaled[i]
    y_fixed_point = int(y_coeff * (2**16)) if abs(y_coeff) > 1e-15 else 0
    if y_fixed_point < 0:
        y_fixed_point = y_fixed_point & 0xFFFFFFFF
    y_hex_str = f"{y_fixed_point:x}" if y_fixed_point != 0 else "0"
    
    csv_data.append([x_hex_str, y_hex_str])

# 保存为CSV文件
with open('distortion_coefficients.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(csv_data)

print("系数已保存到 distortion_coefficients.csv")
print("文件格式：15行2列")
print("左列：X方向系数 (a0-a14)")
print("右列：Y方向系数 (b0-b14)")
print("数据格式：16进制小写，0省略前导零")

系数已保存到 distortion_coefficients.csv
文件格式：15行2列
左列：X方向系数 (a0-a14)
右列：Y方向系数 (b0-b14)
数据格式：16进制小写，0省略前导零
